# Exemple d'utilisation de l'outil AutoGen


## Importer les packages nécessaires


In [ ]:
import os
import json

import requests
from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_core.tools import FunctionTool
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from typing import Any, Callable, Set, Dict, List, Optional

## Création du client

Dans cet exemple, nous utiliserons [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) pour accéder au LLM.

Le `model` est défini comme `gpt-4o-mini`. Essayez de changer le modèle pour un autre disponible sur le marketplace de GitHub Models afin de voir les différents résultats.

Pour un test rapide, nous allons simplement exécuter une invite simple - `Quelle est la capitale de la France`.


In [ ]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## Définir les fonctions

Dans cet exemple, nous allons donner à l'agent accès à un outil qui est une fonction contenant une liste de destinations de vacances disponibles ainsi que leur disponibilité.

Vous pouvez imaginer qu'il s'agit d'un scénario où un agent de voyage pourrait avoir accès à une base de données de voyages, par exemple.

Au fur et à mesure que vous parcourez cet exemple, n'hésitez pas à essayer de définir de nouvelles fonctions et outils que l'agent peut utiliser.


In [ ]:
from typing import Dict, List, Optional


def vacation_destinations(city: str) -> tuple[str, str]:
    """
    Checks if a specific vacation destination is available
    
    Args:
        city (str): Name of the city to check
        
    Returns:
        tuple: Contains city name and availability status ('Available' or 'Unavailable')
    """
    destinations = {
        "Barcelona": "Available",
        "Tokyo": "Unavailable",
        "Cape Town": "Available",
        "Vancouver": "Available",
        "Dubai": "Unavailable",
    }

    if city in destinations:
        return city, destinations[city]
    else:
        return city, "City not found"

# Example usage:
# city, status = vacation_destinations("Barcelona")
# print(f"How about visiting {city}? It's currently {status} there!")

## Définir l'outil de fonction  
Pour que l'agent utilise `vacation_destinations` comme un `FunctionTool`, nous devons le définir comme tel.  

Nous fournirons également une description de l'outil, ce qui est utile pour que l'agent identifie à quoi cet outil sert par rapport à la tâche demandée par l'utilisateur.


In [ ]:
get_vacations = FunctionTool(
    vacation_destinations, description="Search for vacation destinations and if they are available or not."
)

## Définir l'agent

Nous pouvons maintenant créer l'agent dans le code ci-dessous. Nous définissons le `system_message` pour donner à l'agent des instructions sur la manière d'aider les utilisateurs à trouver des destinations de vacances.

Nous activons également le paramètre `reflect_on_tool_use` en le réglant sur true. Cela permet à l'LLM de prendre la réponse de l'appel de l'outil et de l'envoyer en langage naturel.

Vous pouvez régler ce paramètre sur false pour observer la différence.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[get_vacations],
    system_message="You are a travel agent that helps users find vacation destinations.",
    reflect_on_tool_use=True,
)

## Exécution de l'agent

Nous pouvons maintenant exécuter l'agent avec le message initial de l'utilisateur demandant de faire un voyage à Tokyo.

Vous pouvez changer cette destination de ville pour voir comment l'agent répond à la disponibilité de la ville.


In [ ]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="I would like to take a trip to Tokyo", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()


---

**Avertissement** :  
Ce document a été traduit à l'aide du service de traduction automatique [Co-op Translator](https://github.com/Azure/co-op-translator). Bien que nous nous efforcions d'assurer l'exactitude, veuillez noter que les traductions automatisées peuvent contenir des erreurs ou des inexactitudes. Le document original dans sa langue d'origine doit être considéré comme la source faisant autorité. Pour des informations critiques, il est recommandé de recourir à une traduction professionnelle réalisée par un humain. Nous déclinons toute responsabilité en cas de malentendus ou d'interprétations erronées résultant de l'utilisation de cette traduction.
